# Goal 1. Take results from BLAST and get concencus for each contig
# Goal 2. Compare to clusters from K-means.

## Imports

In [1]:
import numpy as np
import pandas as pd
import os

## Parameters

In [10]:
output = 'tbd'
threshold = 0.8

## Take results from BLAST and get concencus for each contig

In [14]:
cluster_0 = './100kb_subset_data/cluster_0/split_contigs/'
cluster_1 = './100kb_subset_data/cluster_1/split_contigs/'
clusters = [cluster_0, cluster_1]
needed = []

#dictionaries
octocoral = ['Dendronephthya gigantea', 'Xenia sp. Carnegie-2017']
dinoflagellates = ['Breviolum minutum Mf 1.05b.01','Symbiodinium microadriaticum','Symbiodinium sp. clade A Y106',
                   'Symbiodinium sp. clade C Y103','Symbiodinium kawagutii','Symbiodinium natans',
                   'Symbiodinium sp. CCMP2592','Symbiodinium sp. KB8','Symbiodinium sp. CCMP2456',
                   'Symbiodinium pilosum','Symbiodinium necroappetens','Cladocopium goreaui']

#for each cluster
for cluster in clusters:
    print(cluster)
    all_contigs = {}
    concencus_dict = {}
    #for file in each cluster
    for file in os.listdir(cluster):
        filename = file.split('.')[0]
        #count_dict has a count of number of each occurence of match
        count_dict = {'Breviolum minutum Mf 1.05b.01':0,'Symbiodinium microadriaticum':0,
                      'Symbiodinium sp. clade A Y106':0,'Symbiodinium sp. clade C Y103':0,'Symbiodinium kawagutii':0,
                      'Symbiodinium natans':0,'Symbiodinium sp. CCMP2592':0,'Symbiodinium sp. KB8':0,
                      'Symbiodinium sp. CCMP2456':0,'Symbiodinium pilosum':0,'Symbiodinium necroappetens':0,
                      'Cladocopium goreaui':0,'Dendronephthya gigantea':0, 'Xenia sp. Carnegie-2017': 0}
        if '.long.tab' in file:
            with open(cluster + file, 'r') as f:
                #print(filename)
                #get header number, get last column
                for line in f:
                    line = line.split(sep = '\t')
                    #increase count for match
                    for entry in count_dict:
                        if entry in line[24].strip('\n'):
                            count_dict[entry]+=1
            all_contigs[filename] = count_dict
            
    #get count of octocorral and dinoflagellates
    #print(len(all_contigs))
    for p in all_contigs:
        o = 0
        d = 0
        for entry in all_contigs[p]:
            #print(entry)
            if entry in dinoflagellates:
                d+=all_contigs[p][entry]
            if entry in octocoral:
                o+=all_contigs[p][entry]
        
        #decide (octocoral/dinoflagellates/mixed/none)
        decision = 'none'
        if d == 0 and o >= 1:
            decision = 'octocoral'
        if d >= 1 and o ==0 :
            decision = 'dinoflagellates'
        if d >= 1 and o >= 1:
            if (d/o) < threshold:
                decision = 'octo'
                #print('octo '+str(d)+' '+str(o))
            else:
                decision = 'mixed'
                #print('mixed '+str(d)+' '+str(o))
        #add decision to concensus dict
        concencus_dict[p] = decision     
        
    #print(concencus_dict)
    
    #print totals of each
    octo=0
    dino=0
    mixed=0
    none=0
    
    for key in concencus_dict:
        if concencus_dict[key] in 'octocoral':
            octo +=1
            needed.append(key)
        if concencus_dict[key] in 'dinoflagellates':
            dino +=1
        if concencus_dict[key] in 'mixed':
            mixed +=1
        if concencus_dict[key] in 'none':
            none +=1
    print('octo: ' + str(octo))
    print('dino: ' + str(dino))
    print('mixed: ' + str(mixed))
    print('none: ' + str(none))


./100kb_subset_data/cluster_0/split_contigs/
octo: 226
dino: 2
mixed: 1
none: 24
./100kb_subset_data/cluster_1/split_contigs/
octo: 110
dino: 85
mixed: 0
none: 221


In [18]:
#write to output file
with open('./out_dir/cluster_info/needed.txt','w') as out:
    for i in needed:
        out.write(str(i) + '\n')
out.close()

In [17]:
print(len(needed))
#print(needed)

336


In [ ]:
#TODO
#Filter mixed bettter --done
#subset octo --done
#sanity check with blobplot --todo
#compare to kmeans --todo
#send subset to Sergio --todo
#take subset of octocorrals from longest to shortest till genome length
#annotation


